# Imports

In [5]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import sklearn
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier

# Train

In [11]:
train_set = pd.read_csv('train/train_encoded.csv')

In [12]:
train_set.drop('id', axis=1, inplace=True)

In [13]:
train_set

,len_keyword,len_location,len_text,len_text_original,diff_len_text,keyword_sum,keyword_min,keyword_max,keyword_mean,keyword_median,...,location_min,location_max,location_mean,location_median,text_sum,text_min,text_max,text_mean,text_median,target
0,0.0,0.0,43,69,26,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.745943,0.224894,0.474996,0.343243,0.323476,1
1,0.0,0.0,36,38,2,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.545323,0.211650,0.508967,0.363618,0.340166,1
2,0.0,0.0,72,133,61,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,3.925010,0.153758,0.585124,0.327084,0.256037,1
3,0.0,0.0,50,65,15,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.515355,0.150085,0.574226,0.359336,0.346962,1
4,0.0,0.0,52,88,36,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.937988,0.247769,0.444024,0.326443,0.287533,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0.0,0.0,45,83,38,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.803465,0.284299,0.418945,0.350433,0.364511,1
7609,0.0,0.0,82,125,43,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,3.228988,0.190353,0.425825,0.293544,0.279435,1
7610,0.0,0.0,55,65,10,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,4.698983,0.366487,0.629740,0.522109,0.629740,1
7611,0.0,0.0,96,137,41,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,4.510709,0.157250,0.529638,0.265336,0.228318,1


In [14]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]

## Bagging

In [15]:
lgbm_model = lgb.LGBMClassifier()

model = BaggingClassifier(base_estimator=lgbm_model)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
results = model_selection.cross_val_score(model, X, y, cv=cv, scoring='f1')

print(results.mean())

0.5798494083406762


In [16]:
lgbm_model = lgb.LGBMClassifier(objective='binary', num_leaves=40, learning_rate=0.1, max_depth=100)

model = BaggingClassifier(base_estimator=lgbm_model)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
results = model_selection.cross_val_score(model, X, y, cv=cv, scoring='f1')

print(results.mean())

0.5859327091674349


In [17]:
lgbm_model = lgb.LGBMClassifier(objective='binary', num_leaves=40, learning_rate=0.1, max_depth=100)

model = BaggingClassifier(base_estimator=lgbm_model)

cv = RepeatedStratifiedKFold(random_state=42)
results = model_selection.cross_val_score(model, X, y, cv=cv, scoring='f1')

print(results.mean())

0.5848855409293382


In [18]:
seed = 7
num_trees = 100
kfold = model_selection.KFold(n_splits=2)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)

results = model_selection.cross_val_score(model, X, y, cv=kfold)
print(results.mean())

0.6072499203212933


In [19]:
kfold = model_selection.KFold(n_splits=3)
estimators = []

model1 = lgb.LGBMClassifier(objective='binary', num_leaves=20, learning_rate=0.1)
estimators.append(('lgbm', model1))

model2 = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.01, max_depth = 100, n_estimators = 100)
estimators.append(('xgb', model2))

model3 = CatBoostClassifier(silent=True)
estimators.append(('catboost', model3))

model4 = AdaBoostClassifier(n_estimators=100, random_state=42)
estimators.append(('adaboost', model4))

ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X, y, cv=kfold)
print(results.mean())

0.6404863911146811


0.6081618329246296

In [20]:
kfold = model_selection.KFold(n_splits=2)
estimators = []

model1 = lgb.LGBMClassifier()
estimators.append(('lgbm', model1))

model2 = xgb.XGBClassifier()
estimators.append(('xgb', model2))

model3 = CatBoostClassifier(silent=True)
estimators.append(('catboost', model3))

ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X, y, cv=kfold)
print(results.mean())

0.626690869116975


In [21]:
kfold = model_selection.KFold(n_splits=10)
estimators = []


model4 = AdaBoostClassifier(n_estimators=100, random_state=42)
estimators.append(('adaboost', model4))

model1 = lgb.LGBMClassifier(objective='binary', num_leaves=20, learning_rate=0.1)
estimators.append(('lgbm', model1))

model3 = CatBoostClassifier(silent=True)
estimators.append(('catboost', model3))

model2 = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.01, max_depth = 100, n_estimators = 100)
estimators.append(('xgb', model2))

ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X, y, cv=kfold)
print(results.mean())

0.6490122128295066
